# Lectura de los datos 

## Task 1.1
Leer el archivo CSV proporcionado y almacenarlo en un np.array para ser trabajado en el notebook.

In [7]:
import numpy as np
from math import *
import random as rd
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [8]:
data = pd.read_csv("./framingham.csv")
data = data.dropna()

## Task 1.2
Ajustar un modelo logístico polinomial en base al juego de datos cargado de forma matricial que relaciona las
variables independientes que usted considere apropiadas (puede no utilizar todas las componentes de X), con la
variable dependiente de salida (sufre o no sufre un paro cardíaco)

In [15]:
paroCardiaco = data['TenYearCHD']
paroCardiaco.value_counts()

data.isna().sum()


male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

### Normalizacion y escalacion de los datos

In [ ]:
columns_to_normalize = ['cigsPerDay', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'heartRate', 'glucose']

scaler = StandardScaler()

scaled_data = scaler.fit_transform(data[columns_to_normalize])
df_scaled = pd.DataFrame(scaled_data, columns=columns_to_normalize)
data = pd.concat([data.drop(columns_to_normalize, axis=1), df_scaled], axis=1)

data.head()

###PCA

In [ ]:

data = data.dropna()

y = data['TenYearCHD']
y = y.to_numpy()
X = data.drop('TenYearCHD', axis=1)

pca = PCA(n_components=7)
X = pca.fit_transform(X)